In [45]:
import numpy as np
import pandas as pd
from collections import Counter
# Counter outil pour effectuer rapidement et facilement des dénombrements.
data = pd.read_csv("results.csv" , sep=',')
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [48]:
# Number of Matches per Year
splitted_date = data['date'].str.split('-')
# creating year column
data['year'] = [x[0] for x in splitted_date]
# creating month column
data['month'] = [x[1] for x in splitted_date]
# get the number of matches per year
year_counts = data['year'].value_counts()
year_counts.head()

2019    1155
2008    1091
2011    1081
2004    1064
2000    1026
Name: year, dtype: int64

In [49]:
# find the years with the largest and the smallest number of matches
year_counts.sort_values(ascending=False).iloc[[0, -1]]

2019    1155
1875       1
Name: year, dtype: int64

In [50]:
# get the number of matches in each month of the year
month_counts = data['month'].value_counts()
# find the months with the largest and the smallest number of matches
month_counts.sort_values(ascending=False).iloc[[0, -1]]

06    5969
01    2218
Name: month, dtype: int64

In [75]:
# create a new column 
data['total_goals'] = data['home_score'] + data['away_score']
# game with the largest score
data[data['total_goals'] == data['total_goals'].max()]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,total_goals
23796,2001-04-11,Australia,American Samoa,31,0,FIFA World Cup qualification,Coffs Harbour,Australia,False,2001,04,31


In [76]:
# Number of Goals per Year
year_goals = data.groupby('year')['total_goals'].sum()
year_goals.sort_values(ascending=False).head()

year
2019    3318
2001    3101
2004    3098
2011    2980
2008    2950
Name: total_goals, dtype: int64

In [64]:
# Team that played the Largest Number of Matches
tournaments_of_interest = ['Friendly', 'FIFA World Cup', 'UEFA Euro', 'African Cup of Nations', 'CFU Caribbean Cup', 
                           'Copa América', 'AFC Asian Cup', 'Other']
cdf1 = data[['home_team', 'tournament']].rename(columns={'home_team': 'team'})
cdf2 = data[['away_team', 'tournament']].rename(columns={'away_team': 'team'})
cdf = pd.concat([cdf1, cdf2], axis=0, ignore_index=True)
cdf['count'] = 1
cdf_piv = cdf.pivot_table(values='count', index='team', columns='tournament', aggfunc=np.sum, fill_value=0)
tournament_counts = cdf['tournament'].value_counts(sort=True, ascending=False)
teams = data['home_team'].append(data['away_team'], ignore_index=True)
team_counts = teams.value_counts().sort_values(ascending=False)
team_counts.head()

Sweden       1020
England      1011
Brazil        981
Argentina     980
Germany       951
dtype: int64

In [67]:
# Team that scored the largest number of goals
cdf1 = data[['home_team', 'home_score']].rename(columns={'home_team': 'team', 'home_score': 'score'})
cdf2 = data[['away_team', 'away_score']].rename(columns={'away_team': 'team', 'away_score': 'score'})
cdf = pd.concat([cdf1, cdf2], axis=0, ignore_index=True)
cdf = cdf.groupby('team').sum().sort_values(by='score', axis=0, ascending=False).head(20)
cdf.head()

,score
team,
England,2209
Brazil,2149
Germany,2126
Sweden,2015
Hungary,1889


In [68]:
# Team that scored the largest number of goals in a WC
cdf1 = data[data['tournament'] == 'FIFA World Cup'][['home_team', 'home_score']].rename(
    columns={'home_team': 'team', 'home_score': 'score'})
cdf2 = data[data['tournament'] == 'FIFA World Cup'][['away_team', 'away_score']].rename(
    columns={'away_team': 'team', 'away_score': 'score'})
cdf = pd.concat([cdf1, cdf2], axis=0, ignore_index=True)
cdf = cdf.groupby('team').sum().sort_values(by='score', axis=0, ascending=False).head(20)
display(cdf.head())

,score
team,
Brazil,229
Germany,226
Argentina,137
Italy,128
France,120


In [65]:
#  Teams that played against each other the most
teams = []
for i in range(data.shape[0]):
    tuple_teams = [data.loc[i, 'home_team'], data.loc[i, 'away_team']]
    tuple_teams.sort()
    teams.append(tuple(tuple_teams))
common_two_teams = Counter(teams).most_common(20)
common_two_teams = [(' VS '.join(a), b) for a,b in common_two_teams]
common_two_teams = pd.DataFrame(common_two_teams, columns=['Teams', 'No. of matches'])
common_two_teams.head()

In [22]:
# Team that won the Largest Number of Matches
def getMatchScore(scores):
    if scores['home_score'] > scores['away_score']:
        return 'home_win'
    elif scores['home_score'] == scores['away_score']:
        return 'draw'
    else:
        return 'away_win'

data['result'] = data[['home_score', 'away_score']].apply(getMatchScore, axis=1)

def getWinner(s):
    if s['result'] == 'draw':
        return 'draw'
    elif s['result'] == 'home_win':
        return s['home_team']
    else:
        return s['away_team']
        
data['winner'] = data[['home_team', 'away_team', 'result']].apply(getWinner, axis=1)

In [6]:
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,result,winner
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,draw,draw
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,home_win,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,home_win,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,draw,draw
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,home_win,Scotland
